<a href="https://colab.research.google.com/github/KenObata/TISMIR_notebooks/blob/main/week15_BERT_base_keras_KFold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook uses Pre-Trained BERT model

Situation: English only (=multi-class).
Split: StratifiedKfold.


### set up

In [ ]:
!pip3 install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 6.3 MB/s 


In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from collections import Counter

from skmultilearn.model_selection import IterativeStratification   
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import csr_matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import balanced_accuracy_score
DIR = '/content/drive/MyDrive/music4all/'
def get_balanced_accuracy(model, McNemar, is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning):
  test_y = test.map(map_func_only_y)
  y_category=np.zeros((TEST_SIZE, ))
  counter=0
  for label_tensor in test_y.take(len(test_y)):
    y_test = np.argmax(label_tensor, axis=1)
    for label in y_test:
      y_category[counter]=label
      counter+=1

  X_test, y_test = test.map(map_func_only_X), y_category
  y_predict_test = np.asarray(model.predict(X_test))
  y_predict_test = np.argmax(y_predict_test, axis=1)
  print(classification_report(y_test, y_predict_test) )
  print(balanced_accuracy_score(y_test, y_predict_test))

  McNemar[(is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning)] = []
  for ground_truh, pred in zip(y_test, y_predict_test):
        if ground_truh==pred:
          McNemar[(is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning)].append(True)
        else:
          McNemar[(is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning)].append(False)
  with open(DIR+ "BERT_base_log.txt", "a") as f:
    print("======================================", file=f)
    print("is_fine_tuning?:", is_fine_tuning, "drop_out_rate: ", drop_out_rate, "learning_rate_transfer_learning: ", learning_rate_transfer_learning,
          "learning_rate_fine_tuning: ", learning_rate_fine_tuning, file=f)
    print(classification_report(y_test, y_predict_test) , file=f)
    print(balanced_accuracy_score(y_test, y_predict_test), file=f)

  return balanced_accuracy_score(y_test, y_predict_test), McNemar

In [ ]:
#save output into text
DIR = '/content/drive/MyDrive/music4all/'
with open(DIR+ "BERT_base_log.txt", "a") as f:
  print("File name: week15_BERT_baseline_keras.ipynb", file=f)


### Data Preparation(Kfold split)

Create dataframe for Gensim

In [ ]:
import pandas as pd

DIR = '/content/drive/MyDrive/music4all/'
df_genre_by_lang = pd.read_csv(DIR + 'df_genre_by_lang_full.csv')
df_genre_by_lang

,Unnamed: 0,id,genres,lang,lyric,number_of_line
0,0,0009fFIM1eYThaPg,pop,en,a sunny day so I got nowhere to hide Not a clo...,91
1,1,00P2bHdWFkghmDqz,soul,en,Tell me a tale that always was Sing me a song ...,36
2,2,00b6fV3nx5z2b8Ls,pop,en,A buh A buh You went to school to learn girl T...,74
3,3,013QDoTqbexEwkHr,pop,en,like a conversation where stops to breathe Is ...,20
4,4,01EKNot8qVgZpKM7,rock,en,Say the words I cannot say Say them on another...,31
...,...,...,...,...,...,...
13535,13535,zzT504Z94j1IAuc3,indie rock,en,think what afraid of come in you know been mad...,18
13536,13536,zzgS4ZqyswamEWNj,pop,en,Oh yeah yeah Last night I took a walk in the s...,75
13537,13537,zzx8CWdM7qkxKQpC,indie rock,en,Innocence it come easy in a sense it never wil...,34
13538,13538,zzz0n04uuTUA7fNh,pop,en,Girl you know how I feel I really Since you be...,65


In [ ]:
def load_data(df_col, y):
    texts, labels = [], []
    
    for line in df_col:
        # texts are already tokenized, just split on space
        # in a real use-case we would put more effort in preprocessing
        texts.append(line.split(' '))
    return pd.DataFrame({'texts': texts, 'labels': y})

In [ ]:
data = load_data(df_genre_by_lang["lyric"], df_genre_by_lang["genres"])

In [ ]:
data

,texts,labels
0,"[a, sunny, day, so, I, got, nowhere, to, hide,...",pop
1,"[Tell, me, a, tale, that, always, was, Sing, m...",soul
2,"[A, buh, A, buh, You, went, to, school, to, le...",pop
3,"[like, a, conversation, where, stops, to, brea...",pop
4,"[Say, the, words, I, cannot, say, Say, them, o...",rock
...,...,...
13535,"[think, what, afraid, of, come, in, you, know,...",indie rock
13536,"[Oh, yeah, yeah, Last, night, I, took, a, walk...",pop
13537,"[Innocence, it, come, easy, in, a, sense, it, ...",indie rock
13538,"[Girl, you, know, how, I, feel, I, really, Sin...",pop


In [ ]:
data['labels'] = data['labels'].astype('category')
label_mapping = data['labels'].cat.categories
data['labels'] = data['labels'].cat.codes
X = data['texts']
y = data['labels']

In [ ]:
type(X)

pandas.core.series.Series

In [ ]:
def StratifiedKFold_feature_and_df_glove(df, feature_list, y_name):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1209)  # 20% for test set 
  y = df[y_name]
  skf.get_n_splits(df[ feature_list ], y)

  splits = []

  for train_index, test_index in skf.split(df[ feature_list ], y):
      print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = df[ feature_list ].loc[train_index], df[ feature_list ].loc[test_index]
      y_train, y_test = y.loc[train_index], y.loc[test_index]
      splits.append({'X_train': X_train, 'X_test': X_test, 'y_train':y_train, 'y_test':y_test })
  return splits

In [ ]:
def StratifiedKFold_feature_and_df(X, y):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1209)  # 20% for test set 
  #y = df[y_name]
  skf.get_n_splits(X, y)#df[ feature_list ]

  splits = []

  for train_index, test_index in skf.split(X, y):#df[ feature_list ]
      print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = X.loc[train_index], X.loc[test_index]
      y_train, y_test = y.loc[train_index], y.loc[test_index]
      splits.append({'X_train': X_train, 'X_test': X_test, 'y_train':y_train, 'y_test':y_test })
  return splits

In [ ]:
#feature_list = ["texts"] #this is BOW and TF-IDF
#splits = StratifiedKFold_feature_and_df( data, feature_list, 'labels')
splits = StratifiedKFold_feature_and_df( X,y)

TRAIN: [    0     1     3 ... 13537 13538 13539] TEST: [    2     4     5 ... 13526 13532 13535]
TRAIN: [    0     2     4 ... 13535 13536 13539] TEST: [    1     3     7 ... 13530 13537 13538]
TRAIN: [    0     1     2 ... 13537 13538 13539] TEST: [    8    14    22 ... 13521 13531 13536]
TRAIN: [    0     1     2 ... 13537 13538 13539] TEST: [   10    12    15 ... 13523 13525 13534]
TRAIN: [    1     2     3 ... 13536 13537 13538] TEST: [    0     6    11 ... 13529 13533 13539]


In [ ]:
len(splits)

5

In [ ]:
split0=splits[0]
print(split0['X_train'].shape)
print(split0['y_train'].shape)
print(split0['X_test'].shape)
print(split0['y_test'].shape)

(10832,)
(10832,)
(2708,)
(2708,)


In [ ]:
split0['X_train']

0        [a, sunny, day, so, I, got, nowhere, to, hide,...
1        [Tell, me, a, tale, that, always, was, Sing, m...
3        [like, a, conversation, where, stops, to, brea...
6        [Locked, up, tight, Like, I, would, never, fee...
7        [sittin, in, the, crib, dreamin, about, leer, ...
                               ...                        
13534    [grandma, cookies, nigga, Shout, out, to, fron...
13536    [Oh, yeah, yeah, Last, night, I, took, a, walk...
13537    [Innocence, it, come, easy, in, a, sense, it, ...
13538    [Girl, you, know, how, I, feel, I, really, Sin...
13539    [wwI, oh, must, go, on, standing, You, break, ...
Name: texts, Length: 10832, dtype: object

In [ ]:
split0['y_train']

0        4
1        9
3        4
6        4
7        6
        ..
13534    6
13536    4
13537    3
13538    4
13539    8
Name: labels, Length: 10832, dtype: int8

### Use my self programmed balanced accuracy

In [ ]:
metrics = Metrics()
history = model.fit(train, validation_data=val, epochs=10, class_weight=my_weight ,callbacks=[metrics])
metrics.get_data()

Epoch 1/10
339/339 [==============================] - 225s 645ms/step - loss: 0.5565 - categorical_accuracy: 0.3878 - val_loss: 1.9337 - val_categorical_accuracy: 0.3273
Epoch 2/10
339/339 [==============================] - 218s 645ms/step - loss: 0.5397 - categorical_accuracy: 0.3626 - val_loss: 2.0651 - val_categorical_accuracy: 0.2764
Epoch 3/10
339/339 [==============================] - 218s 645ms/step - loss: 0.5375 - categorical_accuracy: 0.3392 - val_loss: 2.0931 - val_categorical_accuracy: 0.2273
Epoch 4/10
339/339 [==============================] - 218s 645ms/step - loss: 0.5378 - categorical_accuracy: 0.3197 - val_loss: 2.0576 - val_categorical_accuracy: 0.2459
Epoch 5/10
339/339 [==============================] - 219s 645ms/step - loss: 0.5333 - categorical_accuracy: 0.3077 - val_loss: 2.0292 - val_categorical_accuracy: 0.2816
Epoch 6/10
339/339 [==============================] - 219s 646ms/step - loss: 0.5373 - categorical_accuracy: 0.2967 - val_loss: 2.1738 - val_categoric

[{'balanced_accuracy': 0.09744849916749872},
 {'balanced_accuracy': 0.10420636932765996},
 {'balanced_accuracy': 0.10970405736872939},
 {'balanced_accuracy': 0.10356561320774078},
 {'balanced_accuracy': 0.10792864804613309},
 {'balanced_accuracy': 0.09168989857066105},
 {'balanced_accuracy': 0.09926135499834263},
 {'balanced_accuracy': 0.10014630125778516},
 {'balanced_accuracy': 0.09807505457274496},
 {'balanced_accuracy': 0.09881853716030842}]

## From here, separate X_train, X_test from KFOldSplit

### Preprocess my lyrics data (Official train and test)

In [ ]:
import tensorflow as tf
!pip3 install transformers
SEQ_LEN = 256#512

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 14.4 MB/s 
     |████████████████████████████████| 120 kB 73.2 MB/s 
     |████████████████████████████████| 6.6 MB 62.7 MB/s 


In [ ]:
split0['X_test']

2        [A, buh, A, buh, You, went, to, school, to, le...
4        [Say, the, words, I, cannot, say, Say, them, o...
5        [I, was, alone, I, was, made, of, stone, You, ...
9        [Again, the, burden, of, losing, rests, upon, ...
20       [only, been, three, weeks, And, a, bag, of, sp...
                               ...                        
13517    [Like, the, legend, of, the, Phoenix, All, end...
13522    [Mr, Telephone, man, something, wrong, with, m...
13526    [can, you, imagine, what, it, would, be, like,...
13532    [Love, of, my, life, hurt, me, broken, my, hea...
13535    [think, what, afraid, of, come, in, you, know,...
Name: texts, Length: 2708, dtype: object

In [ ]:
def prepare_lyrics(X_series):
  for i, token_list in X_series.items():
    if type(token_list) is list:
      X_series.loc[i] = ' '.join(token_list)
  return X_series

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
SEQ_LEN=256

def get_Xid_Xmask(X_origin):
  Xids_train = np.zeros((X_origin.shape[0], SEQ_LEN))
  Xmask_train = np.zeros((X_origin.shape[0], SEQ_LEN))

  for i, lyric in enumerate(X_origin):
    tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
    Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']
  return Xids_train, Xmask_train

In [ ]:
def map_func(input_ids, masks, labels):
  return {'input_ids': input_ids, 'attention_mask':masks}, labels

In [ ]:
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )
#tokenizer = AlbertTokenizer.from_pretrained('albert-large-v2')
Xids_train, Xmask_train = get_Xid_Xmask(split0['X_train'])
Xids_test, Xmask_test = get_Xid_Xmask(split0['X_test'])

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))
dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)

dataset_train = dataset_train.shuffle(42).batch(16)
DS_LEN = len(list(dataset_train))
SPLIT=0.9
train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(16)

In [ ]:
counter = Counter(split0['y_train'])
SUM=0
for item in list(counter.values()) :
  SUM+=item
#SUM = sum(counter.values())
SUM

10832

In [ ]:
#Tutorial
#weight_for_0 = (1 / neg) * (total / 2.0)
#weight_for_1 = (1 / pos) * (total / 2.0)

counter = Counter(split0['y_train'])
my_weight2 = {}
print(counter)

for genre in counter:
  #print(genre, counter[genre])
  my_weight2[genre] = (1/counter[genre]) * (SUM/10)
my_weight2

Counter({4: 4143, 7: 1159, 9: 1030, 3: 865, 6: 783, 0: 763, 1: 690, 8: 556, 2: 537, 5: 306})


{4: 0.2614530533429882,
 9: 1.051650485436893,
 6: 1.383397190293742,
 2: 2.0171322160148977,
 0: 1.4196592398427261,
 7: 0.9345987920621226,
 1: 1.569855072463768,
 5: 3.539869281045752,
 8: 1.948201438848921,
 3: 1.2522543352601156}

In [ ]:
TEST_SIZE = len(split0['X_test'])

In [ ]:
def map_func_only_X(val_dictionary, labels):
  return {'input_ids': val_dictionary['input_ids'], 'attention_mask':val_dictionary['attention_mask']}

In [ ]:
def map_func_only_y(val_dictionary, labels):
  return labels

### First Fold Do parameter tuning for dropout rate

In [ ]:
from transformers import AlbertTokenizer, TFAlbertModel
import numpy as np

split0=splits[0]
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )
SEQ_LEN=256
print(split0['X_train'].shape, split0['X_test'].shape)

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Xids_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xmask_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']

for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']

print("Xids_train.shape, Xids_test.shape: ",Xids_train.shape, Xids_test.shape)

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))

dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)
dataset_train = dataset_train.shuffle(42).batch(16)

DS_LEN = len(list(dataset_train))
SPLIT = 0.9
train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(16)

(10832,) (2708,)
Xids_train.shape, Xids_test.shape:  (10832, 256) (2708, 256)


I have intentionally separated cells for check point purpose, based on dropout rates

In [ ]:
seen_parameter = []#(drop_out_rate, initial_rate, fine_tune_learning_rate)

In [ ]:
McNemar={}
balanced_accuracies_transfer_learning=[]
balanced_accuracies_fine_tuning = []

In [ ]:
del(model1)

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.1
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = bert_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    #model2.summary()

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.1


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 96s 135ms/step - loss: 2.1319 - categorical_accuracy: 0.2347 - val_loss: 2.0921 - val_categorical_accuracy: 0.2105
Epoch 2/6
609/609 [==============================] - 80s 131ms/step - loss: 1.9408 - categorical_accuracy: 0.2729 - val_loss: 2.0875 - val_categorical_accuracy: 0.2022
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 1.8647 - categorical_accuracy: 0.2935 - val_loss: 2.0850 - val_categorical_accuracy: 0.2077
Epoch 4/6
609/609 [==============================] - 79s 130ms/step - loss: 1.8166 - categorical_accuracy: 0.3032 - val_loss: 2.0649 - val_categorical_accuracy: 0.2371
Epoch 5/6
609/609 [==============================] - 79s 130ms/step - loss: 1.7702 - categorical_accuracy: 0.3107 - val_loss: 2.0454 - val_categorical_accuracy: 0.2417
Epoch 6/6
609/609 [==============================] - 80s 131ms/step - loss: 1.7220 - categorical_accuracy: 0.3264 - val_loss: 2.0349 - val_categorical_accuracy:

609/609 [==============================] - 203s 306ms/step - loss: 1.6609 - categorical_accuracy: 0.3315 - val_loss: 2.0087 - val_categorical_accuracy: 0.2390
Epoch 2/6
609/609 [==============================] - 184s 302ms/step - loss: 1.4871 - categorical_accuracy: 0.3785 - val_loss: 2.0253 - val_categorical_accuracy: 0.2574
Epoch 3/6
609/609 [==============================] - 184s 302ms/step - loss: 1.3338 - categorical_accuracy: 0.4262 - val_loss: 1.9874 - val_categorical_accuracy: 0.2638
Epoch 4/6
609/609 [==============================] - 184s 302ms/step - loss: 1.2325 - categorical_accuracy: 0.4674 - val_loss: 1.9207 - val_categorical_accuracy: 0.3015
Epoch 5/6
609/609 [==============================] - 184s 302ms/step - loss: 1.1124 - categorical_accuracy: 0.5087 - val_loss: 1.8872 - val_categorical_accuracy: 0.3033
Epoch 6/6
609/609 [==============================] - 184s 302ms/step - loss: 1.0131 - categorical_accuracy: 0.5400 - val_loss: 1.7818 - val_categorical_accuracy: 0.3

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 135ms/step - loss: 2.1473 - categorical_accuracy: 0.2212 - val_loss: 2.2279 - val_categorical_accuracy: 0.1829
Epoch 2/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9332 - categorical_accuracy: 0.2876 - val_loss: 2.1640 - val_categorical_accuracy: 0.2132
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 1.8633 - categorical_accuracy: 0.3019 - val_loss: 2.1020 - val_categorical_accuracy: 0.2279
Epoch 4/6
609/609 [==============================] - 79s 130ms/step - loss: 1.8000 - categorical_accuracy: 0.3178 - val_loss: 2.1010 - val_categorical_accuracy: 0.2353
Epoch 5/6
609/609 [==============================] - 79s 130ms/step - loss: 1.7647 - categorical_accuracy: 0.3313 - val_loss: 2.0504 - val_categorical_accuracy: 0.2390
Epoch 6/6
609/609 [==============================] - 79s 130ms/step - loss: 1.7099 - categorical_accuracy: 0.3393 - val_loss: 2.1037 - val_categorical_accuracy:

609/609 [==============================] - 203s 307ms/step - loss: 1.6337 - categorical_accuracy: 0.3768 - val_loss: 2.0904 - val_categorical_accuracy: 0.2362
Epoch 2/6
609/609 [==============================] - 184s 302ms/step - loss: 1.5721 - categorical_accuracy: 0.3933 - val_loss: 2.0658 - val_categorical_accuracy: 0.2509
Epoch 3/6
609/609 [==============================] - 184s 302ms/step - loss: 1.5512 - categorical_accuracy: 0.3962 - val_loss: 2.0876 - val_categorical_accuracy: 0.2426
Epoch 4/6
609/609 [==============================] - 184s 302ms/step - loss: 1.5234 - categorical_accuracy: 0.3987 - val_loss: 2.0804 - val_categorical_accuracy: 0.2445
Epoch 5/6
609/609 [==============================] - 184s 302ms/step - loss: 1.4948 - categorical_accuracy: 0.4064 - val_loss: 2.0830 - val_categorical_accuracy: 0.2528
Epoch 6/6
609/609 [==============================] - 184s 302ms/step - loss: 1.4758 - categorical_accuracy: 0.4143 - val_loss: 2.0677 - val_categorical_accuracy: 0.2

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 95s 136ms/step - loss: 2.3780 - categorical_accuracy: 0.1911 - val_loss: 2.3196 - val_categorical_accuracy: 0.1691
Epoch 2/6
609/609 [==============================] - 80s 131ms/step - loss: 2.1655 - categorical_accuracy: 0.2341 - val_loss: 2.2731 - val_categorical_accuracy: 0.1884
Epoch 3/6
609/609 [==============================] - 80s 131ms/step - loss: 2.0569 - categorical_accuracy: 0.2648 - val_loss: 2.2394 - val_categorical_accuracy: 0.2022
Epoch 4/6
 50/609 [=>............................] - ETA: 1:06 - loss: 1.9891 - categorical_accuracy: 0.2837

KeyboardInterrupt: ignored

Resume from the last cell

In [ ]:
seen_parameter.append((0.1, 1e-3, 1e-5))
seen_parameter.append((0.1, 1e-3, 1e-6))
seen_parameter

[(0.1, 0.001, 1e-05), (0.1, 0.001, 1e-06)]

In [ ]:
seen_parameter.remove((0.1, [0.001, 0.0001], [1e-05, 1e-06]))
seen_parameter

[(0.1, 0.001, 1e-05), (0.1, 0.001, 1e-06)]

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.1
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    if (drop_out_rate, learning_rate_transfer_learning, learning_rate_fine_tuning) not in seen_parameter:
      seen_parameter.append( (drop_out_rate, learning_rate_transfer_learning, learning_rate_fine_tuning) )
      print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
      , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
      #step1
      bert_base = TFAutoModel.from_pretrained('bert-base-cased')
      input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
      mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

      embeddings = bert_base(input_ids, attention_mask= mask)[0]
      X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
      X = tf.keras.layers.BatchNormalization()(X)
      X = tf.keras.layers.Dense(128, activation = 'relu')(X)
      X = tf.keras.layers.Dropout(drop_out_rate)(X)
      X = tf.keras.layers.Dense(32, activation = 'relu')(X)
      y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

      model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
      model1.layers[2].trainable = False
      #model2.summary()

      #step2
      optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
      loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
      metrics = []
      metrics.append(
          tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
      model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
      #model2.summary() #Check trainable params increased.

      #step3: transfer learning
      early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
      history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

      #step4: predict
      balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
      balanced_accuracies_transfer_learning.append( balanced_acc )

      #step5: fine tune
      print("Fine tuning---------------")
      model1.layers[2].trainable = True

      # It's important to recompile your model after you make any changes
      optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
      loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
      metrics = []
      metrics.append(
          tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
      model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

      history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
      balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
      balanced_accuracies_fine_tuning.append( balanced_acc )
      print("----------------------------------------")
      del(model1)


learning_rate_transfer_learning:  0.0001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.1


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 134ms/step - loss: 2.3379 - categorical_accuracy: 0.1751 - val_loss: 2.2345 - val_categorical_accuracy: 0.1792
Epoch 2/6
609/609 [==============================] - 79s 129ms/step - loss: 2.1498 - categorical_accuracy: 0.2226 - val_loss: 2.1989 - val_categorical_accuracy: 0.1820
Epoch 3/6
609/609 [==============================] - 79s 129ms/step - loss: 2.0597 - categorical_accuracy: 0.2433 - val_loss: 2.1829 - val_categorical_accuracy: 0.1958
Epoch 4/6
609/609 [==============================] - 79s 129ms/step - loss: 2.0020 - categorical_accuracy: 0.2677 - val_loss: 2.1751 - val_categorical_accuracy: 0.2151
Epoch 5/6
609/609 [==============================] - 79s 129ms/step - loss: 1.9775 - categorical_accuracy: 0.2714 - val_loss: 2.1667 - val_categorical_accuracy: 0.2142
Epoch 6/6
609/609 [==============================] - 79s 129ms/step - loss: 1.9377 - categorical_accuracy: 0.2838 - val_loss: 2.1869 - val_categorical_accuracy:

609/609 [==============================] - 202s 308ms/step - loss: 1.8963 - categorical_accuracy: 0.2941 - val_loss: 2.0672 - val_categorical_accuracy: 0.2344
Epoch 2/6
609/609 [==============================] - 184s 302ms/step - loss: 1.7437 - categorical_accuracy: 0.3323 - val_loss: 2.0995 - val_categorical_accuracy: 0.2353
Epoch 3/6
609/609 [==============================] - 184s 302ms/step - loss: 1.6397 - categorical_accuracy: 0.3580 - val_loss: 2.0585 - val_categorical_accuracy: 0.2785
Epoch 4/6
609/609 [==============================] - 184s 302ms/step - loss: 1.5056 - categorical_accuracy: 0.3900 - val_loss: 1.9967 - val_categorical_accuracy: 0.2785
Epoch 5/6
609/609 [==============================] - 184s 302ms/step - loss: 1.3985 - categorical_accuracy: 0.4325 - val_loss: 2.0502 - val_categorical_accuracy: 0.2546
Epoch 6/6
609/609 [==============================] - 184s 302ms/step - loss: 1.3024 - categorical_accuracy: 0.4699 - val_loss: 2.0000 - val_categorical_accuracy: 0.2

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 93s 134ms/step - loss: 2.3302 - categorical_accuracy: 0.1521 - val_loss: 2.2585 - val_categorical_accuracy: 0.1710
Epoch 2/6
609/609 [==============================] - 79s 129ms/step - loss: 2.1365 - categorical_accuracy: 0.2204 - val_loss: 2.2393 - val_categorical_accuracy: 0.1884
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 2.0479 - categorical_accuracy: 0.2556 - val_loss: 2.2114 - val_categorical_accuracy: 0.2040
Epoch 4/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9987 - categorical_accuracy: 0.2705 - val_loss: 2.1893 - val_categorical_accuracy: 0.2160
Epoch 5/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9606 - categorical_accuracy: 0.2868 - val_loss: 2.1765 - val_categorical_accuracy: 0.2215
Epoch 6/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9335 - categorical_accuracy: 0.2916 - val_loss: 2.1924 - val_categorical_accuracy:

609/609 [==============================] - 203s 306ms/step - loss: 1.8778 - categorical_accuracy: 0.2889 - val_loss: 2.1768 - val_categorical_accuracy: 0.2169
Epoch 2/6
609/609 [==============================] - 184s 302ms/step - loss: 1.8521 - categorical_accuracy: 0.3012 - val_loss: 2.1548 - val_categorical_accuracy: 0.2142
Epoch 3/6
609/609 [==============================] - 184s 302ms/step - loss: 1.8291 - categorical_accuracy: 0.2991 - val_loss: 2.1823 - val_categorical_accuracy: 0.2151
Epoch 4/6
609/609 [==============================] - 184s 302ms/step - loss: 1.8017 - categorical_accuracy: 0.3156 - val_loss: 2.1226 - val_categorical_accuracy: 0.2325
Epoch 5/6
609/609 [==============================] - 184s 302ms/step - loss: 1.7792 - categorical_accuracy: 0.3224 - val_loss: 2.1187 - val_categorical_accuracy: 0.2362
Epoch 6/6
609/609 [==============================] - 183s 301ms/step - loss: 1.7477 - categorical_accuracy: 0.3291 - val_loss: 2.1155 - val_categorical_accuracy: 0.2

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.2
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = bert_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    #model2.summary()

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.2


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 92s 133ms/step - loss: 2.1491 - categorical_accuracy: 0.2292 - val_loss: 2.1187 - val_categorical_accuracy: 0.2362
Epoch 2/6
609/609 [==============================] - 78s 129ms/step - loss: 1.9675 - categorical_accuracy: 0.2652 - val_loss: 2.1160 - val_categorical_accuracy: 0.2233
Epoch 3/6
609/609 [==============================] - 78s 128ms/step - loss: 1.9018 - categorical_accuracy: 0.2834 - val_loss: 2.1071 - val_categorical_accuracy: 0.2381
Epoch 4/6
609/609 [==============================] - 78s 128ms/step - loss: 1.8360 - categorical_accuracy: 0.3011 - val_loss: 2.0411 - val_categorical_accuracy: 0.2675
Epoch 5/6
609/609 [==============================] - 78s 128ms/step - loss: 1.8013 - categorical_accuracy: 0.2990 - val_loss: 2.0653 - val_categorical_accuracy: 0.2583
Epoch 6/6
609/609 [==============================] - 78s 129ms/step - loss: 1.7748 - categorical_accuracy: 0.3103 - val_loss: 2.0972 - val_categorical_accuracy:

609/609 [==============================] - 202s 306ms/step - loss: 1.7087 - categorical_accuracy: 0.3258 - val_loss: 2.0464 - val_categorical_accuracy: 0.2224
Epoch 2/6
609/609 [==============================] - 184s 302ms/step - loss: 1.5673 - categorical_accuracy: 0.3625 - val_loss: 1.9821 - val_categorical_accuracy: 0.2665
Epoch 3/6
609/609 [==============================] - 184s 302ms/step - loss: 1.4333 - categorical_accuracy: 0.4021 - val_loss: 1.9711 - val_categorical_accuracy: 0.2353
Epoch 4/6
609/609 [==============================] - 184s 302ms/step - loss: 1.3306 - categorical_accuracy: 0.4271 - val_loss: 1.9967 - val_categorical_accuracy: 0.2583
Epoch 5/6
609/609 [==============================] - 184s 302ms/step - loss: 1.2276 - categorical_accuracy: 0.4626 - val_loss: 1.9567 - val_categorical_accuracy: 0.2941
Epoch 6/6
609/609 [==============================] - 183s 301ms/step - loss: 1.1260 - categorical_accuracy: 0.4968 - val_loss: 1.9989 - val_categorical_accuracy: 0.2

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 93s 133ms/step - loss: 2.1620 - categorical_accuracy: 0.2290 - val_loss: 2.1240 - val_categorical_accuracy: 0.2390
Epoch 2/6
609/609 [==============================] - 78s 129ms/step - loss: 1.9556 - categorical_accuracy: 0.2834 - val_loss: 2.1211 - val_categorical_accuracy: 0.2426
Epoch 3/6
609/609 [==============================] - 79s 129ms/step - loss: 1.9017 - categorical_accuracy: 0.2955 - val_loss: 2.0691 - val_categorical_accuracy: 0.2472
Epoch 4/6
581/609 [===========================>..] - ETA: 3s - loss: 1.8363 - categorical_accuracy: 0.3096

Resume from the last

In [ ]:
seen_parameters=[]
seen_parameters.append((0.2,1e-3,1e-5))

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.2
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    if (drop_out_rate, learning_rate_transfer_learning, learning_rate_fine_tuning) not in seen_parameters:
      seen_parameters.append( (drop_out_rate, learning_rate_transfer_learning, learning_rate_fine_tuning) )
      print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
      , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
      #step1
      bert_base = TFAutoModel.from_pretrained('bert-base-cased')
      input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
      mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

      embeddings = bert_base(input_ids, attention_mask= mask)[0]
      X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
      X = tf.keras.layers.BatchNormalization()(X)
      X = tf.keras.layers.Dense(128, activation = 'relu')(X)
      X = tf.keras.layers.Dropout(drop_out_rate)(X)
      X = tf.keras.layers.Dense(32, activation = 'relu')(X)
      y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

      model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
      model1.layers[2].trainable = False
      #model2.summary()

      #step2
      optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
      loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
      metrics = []
      metrics.append(
          tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
      model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
      #model2.summary() #Check trainable params increased.

      #step3: transfer learning
      early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
      history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

      #step4: predict
      balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
      balanced_accuracies_transfer_learning.append( balanced_acc )

      #step5: fine tune
      print("Fine tuning---------------")
      model1.layers[2].trainable = True

      # It's important to recompile your model after you make any changes
      optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
      loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
      metrics = []
      metrics.append(
          tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
      model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

      history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
      balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
      balanced_accuracies_fine_tuning.append( balanced_acc )
      print("----------------------------------------")
      del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-06 drop_out_rate: 0.2


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 95s 135ms/step - loss: 2.1516 - categorical_accuracy: 0.2358 - val_loss: 2.1618 - val_categorical_accuracy: 0.2151
Epoch 2/6
609/609 [==============================] - 79s 129ms/step - loss: 1.9699 - categorical_accuracy: 0.2759 - val_loss: 2.1554 - val_categorical_accuracy: 0.2132
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9137 - categorical_accuracy: 0.2942 - val_loss: 2.0845 - val_categorical_accuracy: 0.2316
Epoch 4/6
609/609 [==============================] - 79s 130ms/step - loss: 1.8559 - categorical_accuracy: 0.2944 - val_loss: 2.1245 - val_categorical_accuracy: 0.2188
Epoch 5/6
609/609 [==============================] - 79s 130ms/step - loss: 1.8044 - categorical_accuracy: 0.3085 - val_loss: 2.1073 - val_categorical_accuracy: 0.2371
Epoch 6/6
609/609 [==============================] - 79s 129ms/step - loss: 1.7854 - categorical_accuracy: 0.3059 - val_loss: 2.0825 - val_categorical_accuracy:

609/609 [==============================] - 202s 305ms/step - loss: 1.7083 - categorical_accuracy: 0.3323 - val_loss: 2.0581 - val_categorical_accuracy: 0.2472
Epoch 2/6
609/609 [==============================] - 183s 301ms/step - loss: 1.6699 - categorical_accuracy: 0.3341 - val_loss: 2.0557 - val_categorical_accuracy: 0.2574
Epoch 3/6
609/609 [==============================] - 183s 301ms/step - loss: 1.6432 - categorical_accuracy: 0.3485 - val_loss: 2.0499 - val_categorical_accuracy: 0.2546
Epoch 4/6
609/609 [==============================] - 183s 301ms/step - loss: 1.6294 - categorical_accuracy: 0.3431 - val_loss: 2.0597 - val_categorical_accuracy: 0.2482
Epoch 5/6
609/609 [==============================] - 183s 301ms/step - loss: 1.6033 - categorical_accuracy: 0.3543 - val_loss: 2.0543 - val_categorical_accuracy: 0.2555
              precision    recall  f1-score   support

         0.0       0.15      0.24      0.18       190
         1.0       0.17      0.56      0.26       173
  

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 135ms/step - loss: 2.3913 - categorical_accuracy: 0.2151 - val_loss: 2.3193 - val_categorical_accuracy: 0.1875
Epoch 2/6
609/609 [==============================] - 79s 130ms/step - loss: 2.1941 - categorical_accuracy: 0.2423 - val_loss: 2.2854 - val_categorical_accuracy: 0.2077
Epoch 3/6
609/609 [==============================] - 79s 129ms/step - loss: 2.1124 - categorical_accuracy: 0.2613 - val_loss: 2.2725 - val_categorical_accuracy: 0.2031
Epoch 4/6
609/609 [==============================] - 79s 129ms/step - loss: 2.0562 - categorical_accuracy: 0.2767 - val_loss: 2.2420 - val_categorical_accuracy: 0.2151
Epoch 5/6
609/609 [==============================] - 79s 129ms/step - loss: 2.0091 - categorical_accuracy: 0.2894 - val_loss: 2.2246 - val_categorical_accuracy: 0.2188
Epoch 6/6
609/609 [==============================] - 79s 129ms/step - loss: 1.9640 - categorical_accuracy: 0.2984 - val_loss: 2.1857 - val_categorical_accuracy:

609/609 [==============================] - 201s 305ms/step - loss: 1.9258 - categorical_accuracy: 0.3061 - val_loss: 2.0286 - val_categorical_accuracy: 0.2849
Epoch 2/6
609/609 [==============================] - 183s 301ms/step - loss: 1.7943 - categorical_accuracy: 0.3421 - val_loss: 2.3040 - val_categorical_accuracy: 0.2105
Epoch 3/6
609/609 [==============================] - 183s 301ms/step - loss: 1.6855 - categorical_accuracy: 0.3704 - val_loss: 2.1378 - val_categorical_accuracy: 0.2739
Epoch 4/6
609/609 [==============================] - 183s 301ms/step - loss: 1.5812 - categorical_accuracy: 0.4054 - val_loss: 2.1208 - val_categorical_accuracy: 0.2583
              precision    recall  f1-score   support

         0.0       0.18      0.24      0.21       190
         1.0       0.14      0.73      0.24       173
         2.0       0.06      0.07      0.07       135
         3.0       0.22      0.13      0.16       216
         4.0       0.71      0.19      0.29      1036
         

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 135ms/step - loss: 2.4167 - categorical_accuracy: 0.1215 - val_loss: 2.3608 - val_categorical_accuracy: 0.1241
Epoch 2/6
609/609 [==============================] - 79s 130ms/step - loss: 2.1944 - categorical_accuracy: 0.1866 - val_loss: 2.2745 - val_categorical_accuracy: 0.1517
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 2.1009 - categorical_accuracy: 0.2241 - val_loss: 2.2354 - val_categorical_accuracy: 0.1590
Epoch 4/6
609/609 [==============================] - 79s 129ms/step - loss: 2.0422 - categorical_accuracy: 0.2475 - val_loss: 2.2182 - val_categorical_accuracy: 0.1737
Epoch 5/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9931 - categorical_accuracy: 0.2672 - val_loss: 2.1976 - val_categorical_accuracy: 0.1801
Epoch 6/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9613 - categorical_accuracy: 0.2813 - val_loss: 2.1860 - val_categorical_accuracy:

609/609 [==============================] - 201s 305ms/step - loss: 1.9227 - categorical_accuracy: 0.2879 - val_loss: 2.1677 - val_categorical_accuracy: 0.1912
Epoch 2/6
609/609 [==============================] - 183s 301ms/step - loss: 1.8981 - categorical_accuracy: 0.2856 - val_loss: 2.1523 - val_categorical_accuracy: 0.1866
Epoch 3/6
609/609 [==============================] - 183s 301ms/step - loss: 1.8679 - categorical_accuracy: 0.3030 - val_loss: 2.1551 - val_categorical_accuracy: 0.1884
Epoch 4/6
609/609 [==============================] - 183s 301ms/step - loss: 1.8315 - categorical_accuracy: 0.3069 - val_loss: 2.1518 - val_categorical_accuracy: 0.1967
Epoch 5/6
609/609 [==============================] - 183s 301ms/step - loss: 1.8190 - categorical_accuracy: 0.3153 - val_loss: 2.1215 - val_categorical_accuracy: 0.2040
Epoch 6/6
609/609 [==============================] - 183s 301ms/step - loss: 1.7975 - categorical_accuracy: 0.3138 - val_loss: 2.1320 - val_categorical_accuracy: 0.1

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.3
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = bert_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    #model2.summary()

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.3


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 135ms/step - loss: 2.1653 - categorical_accuracy: 0.2192 - val_loss: 2.1321 - val_categorical_accuracy: 0.2390
Epoch 2/6
609/609 [==============================] - 79s 129ms/step - loss: 1.9952 - categorical_accuracy: 0.2704 - val_loss: 2.0991 - val_categorical_accuracy: 0.2169
Epoch 3/6
609/609 [==============================] - 79s 129ms/step - loss: 1.9373 - categorical_accuracy: 0.3002 - val_loss: 2.1141 - val_categorical_accuracy: 0.2197
Epoch 4/6
609/609 [==============================] - 79s 130ms/step - loss: 1.8860 - categorical_accuracy: 0.3019 - val_loss: 2.0813 - val_categorical_accuracy: 0.2371
              precision    recall  f1-score   support

         0.0       0.16      0.21      0.18       190
         1.0       0.13      0.47      0.20       173
         2.0       0.01      0.01      0.01       135
         3.0       0.14      0.14      0.14       216
         4.0       0.70      0.22      0.34      1036
   

609/609 [==============================] - 201s 305ms/step - loss: 1.8114 - categorical_accuracy: 0.3343 - val_loss: 2.1019 - val_categorical_accuracy: 0.2243
Epoch 2/6
609/609 [==============================] - 183s 301ms/step - loss: 1.6854 - categorical_accuracy: 0.3645 - val_loss: 2.0400 - val_categorical_accuracy: 0.2518
Epoch 3/6
609/609 [==============================] - 183s 301ms/step - loss: 1.5842 - categorical_accuracy: 0.3850 - val_loss: 1.9871 - val_categorical_accuracy: 0.2730
Epoch 4/6
609/609 [==============================] - 183s 301ms/step - loss: 1.4992 - categorical_accuracy: 0.4056 - val_loss: 1.9411 - val_categorical_accuracy: 0.2950
Epoch 5/6
609/609 [==============================] - 183s 301ms/step - loss: 1.4189 - categorical_accuracy: 0.4220 - val_loss: 1.9245 - val_categorical_accuracy: 0.2904
Epoch 6/6
609/609 [==============================] - 184s 302ms/step - loss: 1.3299 - categorical_accuracy: 0.4457 - val_loss: 2.0824 - val_categorical_accuracy: 0.2

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


...

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 135ms/step - loss: 2.5364 - categorical_accuracy: 0.1238 - val_loss: 2.2467 - val_categorical_accuracy: 0.1452
Epoch 2/6
609/609 [==============================] - 79s 129ms/step - loss: 2.2420 - categorical_accuracy: 0.1767 - val_loss: 2.2205 - val_categorical_accuracy: 0.1645
Epoch 3/6
609/609 [==============================] - 79s 129ms/step - loss: 2.1359 - categorical_accuracy: 0.2172 - val_loss: 2.2164 - val_categorical_accuracy: 0.1691
Epoch 4/6
609/609 [==============================] - 79s 129ms/step - loss: 2.0751 - categorical_accuracy: 0.2328 - val_loss: 2.1977 - val_categorical_accuracy: 0.1921
Epoch 5/6
609/609 [==============================] - 79s 130ms/step - loss: 2.0337 - categorical_accuracy: 0.2496 - val_loss: 2.1751 - val_categorical_accuracy: 0.1921
Epoch 6/6
609/609 [==============================] - 79s 129ms/step - loss: 2.0032 - categorical_accuracy: 0.2621 - val_loss: 2.1710 - val_categorical_accuracy:

609/609 [==============================] - 201s 305ms/step - loss: 1.9329 - categorical_accuracy: 0.2825 - val_loss: 2.0349 - val_categorical_accuracy: 0.2270
Epoch 2/6
609/609 [==============================] - 183s 301ms/step - loss: 1.8183 - categorical_accuracy: 0.3053 - val_loss: 2.0005 - val_categorical_accuracy: 0.2693
Epoch 3/6
609/609 [==============================] - 183s 301ms/step - loss: 1.7409 - categorical_accuracy: 0.3212 - val_loss: 1.8667 - val_categorical_accuracy: 0.3180
Epoch 4/6
609/609 [==============================] - 183s 301ms/step - loss: 1.6407 - categorical_accuracy: 0.3551 - val_loss: 2.1285 - val_categorical_accuracy: 0.2445
Epoch 5/6
609/609 [==============================] - 183s 301ms/step - loss: 1.5492 - categorical_accuracy: 0.3797 - val_loss: 1.7971 - val_categorical_accuracy: 0.3483
Epoch 6/6
609/609 [==============================] - 183s 301ms/step - loss: 1.4567 - categorical_accuracy: 0.4156 - val_loss: 1.9002 - val_categorical_accuracy: 0.3

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 135ms/step - loss: 2.4423 - categorical_accuracy: 0.1418 - val_loss: 2.3235 - val_categorical_accuracy: 0.1498
Epoch 2/6
609/609 [==============================] - 79s 130ms/step - loss: 2.2354 - categorical_accuracy: 0.2040 - val_loss: 2.2644 - val_categorical_accuracy: 0.1737
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 2.1509 - categorical_accuracy: 0.2303 - val_loss: 2.2459 - val_categorical_accuracy: 0.1801
Epoch 4/6
123/609 [=====>........................] - ETA: 57s - loss: 2.1069 - categorical_accuracy: 0.2388

### Resume from the last cell

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.3
learning_rate_transfer_learnings = [ 1e-4]
learning_rate_fine_tunings = [1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = bert_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    #model2.summary()

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.0001 learning_rate_fine_tuning:  1e-06 drop_out_rate: 0.3


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 95s 135ms/step - loss: 2.4495 - categorical_accuracy: 0.1297 - val_loss: 2.3345 - val_categorical_accuracy: 0.1406
Epoch 2/6
609/609 [==============================] - 79s 130ms/step - loss: 2.2148 - categorical_accuracy: 0.1918 - val_loss: 2.2765 - val_categorical_accuracy: 0.1691
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 2.1319 - categorical_accuracy: 0.2072 - val_loss: 2.2452 - val_categorical_accuracy: 0.1765
Epoch 4/6
609/609 [==============================] - 79s 130ms/step - loss: 2.0613 - categorical_accuracy: 0.2320 - val_loss: 2.2178 - val_categorical_accuracy: 0.1921
Epoch 5/6
609/609 [==============================] - 80s 132ms/step - loss: 2.0293 - categorical_accuracy: 0.2409 - val_loss: 2.1922 - val_categorical_accuracy: 0.2022
Epoch 6/6
609/609 [==============================] - 80s 131ms/step - loss: 1.9828 - categorical_accuracy: 0.2560 - val_loss: 2.1917 - val_categorical_accuracy:

609/609 [==============================] - 204s 309ms/step - loss: 1.9627 - categorical_accuracy: 0.2607 - val_loss: 2.1839 - val_categorical_accuracy: 0.1893
Epoch 2/6
609/609 [==============================] - 186s 305ms/step - loss: 1.9341 - categorical_accuracy: 0.2688 - val_loss: 2.1393 - val_categorical_accuracy: 0.2123
Epoch 3/6
609/609 [==============================] - 186s 305ms/step - loss: 1.9075 - categorical_accuracy: 0.2677 - val_loss: 2.1315 - val_categorical_accuracy: 0.2123
Epoch 4/6
609/609 [==============================] - 186s 306ms/step - loss: 1.8853 - categorical_accuracy: 0.2781 - val_loss: 2.1393 - val_categorical_accuracy: 0.2086
Epoch 5/6
609/609 [==============================] - 186s 306ms/step - loss: 1.8665 - categorical_accuracy: 0.2827 - val_loss: 2.1286 - val_categorical_accuracy: 0.2178
Epoch 6/6
609/609 [==============================] - 186s 305ms/step - loss: 1.8527 - categorical_accuracy: 0.2840 - val_loss: 2.1103 - val_categorical_accuracy: 0.2

In [ ]:
seen_parameter = []#(drop_out_rate, initial_rate, fine_tune_learning_rate)

In [ ]:
McNemar={}
balanced_accuracies_transfer_learning=[]
balanced_accuracies_fine_tuning = []

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.4
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = bert_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    #model2.summary()

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.4


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 134ms/step - loss: 2.2317 - categorical_accuracy: 0.1972 - val_loss: 2.1380 - val_categorical_accuracy: 0.1912
Epoch 2/6
609/609 [==============================] - 79s 129ms/step - loss: 2.0176 - categorical_accuracy: 0.2512 - val_loss: 2.1589 - val_categorical_accuracy: 0.2086
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9597 - categorical_accuracy: 0.2723 - val_loss: 2.1580 - val_categorical_accuracy: 0.1930
Epoch 4/6
609/609 [==============================] - 79s 129ms/step - loss: 1.9069 - categorical_accuracy: 0.2855 - val_loss: 2.1173 - val_categorical_accuracy: 0.2491
Epoch 5/6
609/609 [==============================] - 79s 129ms/step - loss: 1.8727 - categorical_accuracy: 0.2930 - val_loss: 2.0645 - val_categorical_accuracy: 0.2408
Epoch 6/6
609/609 [==============================] - 79s 130ms/step - loss: 1.8497 - categorical_accuracy: 0.3043 - val_loss: 2.0751 - val_categorical_accuracy:

609/609 [==============================] - 202s 305ms/step - loss: 1.8155 - categorical_accuracy: 0.2956 - val_loss: 2.0693 - val_categorical_accuracy: 0.2417
Epoch 2/6
609/609 [==============================] - 183s 301ms/step - loss: 1.6760 - categorical_accuracy: 0.3289 - val_loss: 2.0994 - val_categorical_accuracy: 0.2463
Epoch 3/6
609/609 [==============================] - 183s 301ms/step - loss: 1.5883 - categorical_accuracy: 0.3566 - val_loss: 1.9835 - val_categorical_accuracy: 0.2840
Epoch 4/6
609/609 [==============================] - 183s 301ms/step - loss: 1.5029 - categorical_accuracy: 0.3783 - val_loss: 1.9055 - val_categorical_accuracy: 0.2987
Epoch 5/6
609/609 [==============================] - 183s 301ms/step - loss: 1.4270 - categorical_accuracy: 0.4063 - val_loss: 1.9359 - val_categorical_accuracy: 0.3107
Epoch 6/6
609/609 [==============================] - 183s 301ms/step - loss: 1.3398 - categorical_accuracy: 0.4276 - val_loss: 1.8814 - val_categorical_accuracy: 0.3

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 135ms/step - loss: 2.2068 - categorical_accuracy: 0.2057 - val_loss: 2.2205 - val_categorical_accuracy: 0.1461
Epoch 2/6
609/609 [==============================] - 79s 130ms/step - loss: 2.0150 - categorical_accuracy: 0.2399 - val_loss: 2.1568 - val_categorical_accuracy: 0.1930
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9432 - categorical_accuracy: 0.2668 - val_loss: 2.1433 - val_categorical_accuracy: 0.2123
Epoch 4/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9005 - categorical_accuracy: 0.2740 - val_loss: 2.0847 - val_categorical_accuracy: 0.2132
Epoch 5/6
609/609 [==============================] - 79s 130ms/step - loss: 1.8612 - categorical_accuracy: 0.2830 - val_loss: 2.1234 - val_categorical_accuracy: 0.2123
Epoch 6/6
609/609 [==============================] - 80s 131ms/step - loss: 1.8466 - categorical_accuracy: 0.2885 - val_loss: 2.1262 - val_categorical_accuracy:

609/609 [==============================] - 202s 306ms/step - loss: 1.7734 - categorical_accuracy: 0.3100 - val_loss: 2.1188 - val_categorical_accuracy: 0.2142
Epoch 2/6
609/609 [==============================] - 183s 301ms/step - loss: 1.7511 - categorical_accuracy: 0.3136 - val_loss: 2.0763 - val_categorical_accuracy: 0.2371
Epoch 3/6
609/609 [==============================] - 183s 301ms/step - loss: 1.7396 - categorical_accuracy: 0.3172 - val_loss: 2.0898 - val_categorical_accuracy: 0.2233
Epoch 4/6
609/609 [==============================] - 183s 301ms/step - loss: 1.7041 - categorical_accuracy: 0.3252 - val_loss: 2.0878 - val_categorical_accuracy: 0.2353
Epoch 5/6
609/609 [==============================] - 183s 301ms/step - loss: 1.6946 - categorical_accuracy: 0.3273 - val_loss: 2.0737 - val_categorical_accuracy: 0.2261
              precision    recall  f1-score   support

         0.0       0.11      0.21      0.15       190
         1.0       0.15      0.58      0.24       173
  

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 135ms/step - loss: 2.4721 - categorical_accuracy: 0.1269 - val_loss: 2.3431 - val_categorical_accuracy: 0.1369
Epoch 2/6
609/609 [==============================] - 79s 130ms/step - loss: 2.2818 - categorical_accuracy: 0.1688 - val_loss: 2.2817 - val_categorical_accuracy: 0.1452
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 2.1810 - categorical_accuracy: 0.1978 - val_loss: 2.2541 - val_categorical_accuracy: 0.1535
Epoch 4/6
609/609 [==============================] - 79s 130ms/step - loss: 2.1360 - categorical_accuracy: 0.2097 - val_loss: 2.2206 - val_categorical_accuracy: 0.1645
Epoch 5/6
609/609 [==============================] - 79s 130ms/step - loss: 2.0921 - categorical_accuracy: 0.2238 - val_loss: 2.1995 - val_categorical_accuracy: 0.1728
Epoch 6/6
609/609 [==============================] - 79s 130ms/step - loss: 2.0520 - categorical_accuracy: 0.2474 - val_loss: 2.1727 - val_categorical_accuracy:

609/609 [==============================] - 202s 305ms/step - loss: 1.9919 - categorical_accuracy: 0.2618 - val_loss: 1.9537 - val_categorical_accuracy: 0.2895
Epoch 2/6
609/609 [==============================] - 183s 301ms/step - loss: 1.8824 - categorical_accuracy: 0.2885 - val_loss: 2.0117 - val_categorical_accuracy: 0.2675
Epoch 3/6
392/609 [==================>...........] - ETA: 1:02 - loss: 1.7924 - categorical_accuracy: 0.3181

Resume from the last

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.4
learning_rate_transfer_learnings = [1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = bert_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    #model2.summary()

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.0001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.4


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 96s 136ms/step - loss: 2.5474 - categorical_accuracy: 0.1922 - val_loss: 2.2609 - val_categorical_accuracy: 0.1507
Epoch 2/6
609/609 [==============================] - 80s 131ms/step - loss: 2.2904 - categorical_accuracy: 0.1937 - val_loss: 2.2255 - val_categorical_accuracy: 0.1627
Epoch 3/6
609/609 [==============================] - 80s 131ms/step - loss: 2.1950 - categorical_accuracy: 0.2088 - val_loss: 2.2120 - val_categorical_accuracy: 0.1756
Epoch 4/6
609/609 [==============================] - 80s 131ms/step - loss: 2.1501 - categorical_accuracy: 0.2247 - val_loss: 2.1856 - val_categorical_accuracy: 0.1710
Epoch 5/6
609/609 [==============================] - 80s 131ms/step - loss: 2.0989 - categorical_accuracy: 0.2344 - val_loss: 2.1620 - val_categorical_accuracy: 0.1838
Epoch 6/6
609/609 [==============================] - 80s 131ms/step - loss: 2.0665 - categorical_accuracy: 0.2503 - val_loss: 2.1514 - val_categorical_accuracy:

609/609 [==============================] - 205s 308ms/step - loss: 2.0187 - categorical_accuracy: 0.2574 - val_loss: 2.1205 - val_categorical_accuracy: 0.2096
Epoch 2/6
609/609 [==============================] - 185s 304ms/step - loss: 1.9338 - categorical_accuracy: 0.2912 - val_loss: 2.0420 - val_categorical_accuracy: 0.2537
Epoch 3/6
609/609 [==============================] - 185s 304ms/step - loss: 1.8401 - categorical_accuracy: 0.3116 - val_loss: 2.1483 - val_categorical_accuracy: 0.2233
Epoch 4/6
609/609 [==============================] - 185s 304ms/step - loss: 1.7726 - categorical_accuracy: 0.3328 - val_loss: 2.0875 - val_categorical_accuracy: 0.2482
Epoch 5/6
609/609 [==============================] - 185s 304ms/step - loss: 1.6934 - categorical_accuracy: 0.3549 - val_loss: 2.0820 - val_categorical_accuracy: 0.2472
              precision    recall  f1-score   support

         0.0       0.14      0.28      0.19       190
         1.0       0.17      0.44      0.24       173
  

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 135ms/step - loss: 2.4859 - categorical_accuracy: 0.1369 - val_loss: 2.2595 - val_categorical_accuracy: 0.1664
Epoch 2/6
609/609 [==============================] - 79s 130ms/step - loss: 2.2612 - categorical_accuracy: 0.1911 - val_loss: 2.2091 - val_categorical_accuracy: 0.1783
Epoch 3/6
609/609 [==============================] - 80s 131ms/step - loss: 2.1590 - categorical_accuracy: 0.2080 - val_loss: 2.1692 - val_categorical_accuracy: 0.1912
Epoch 4/6
609/609 [==============================] - 80s 131ms/step - loss: 2.1282 - categorical_accuracy: 0.2221 - val_loss: 2.1616 - val_categorical_accuracy: 0.1875
Epoch 5/6
609/609 [==============================] - 80s 131ms/step - loss: 2.0859 - categorical_accuracy: 0.2359 - val_loss: 2.1383 - val_categorical_accuracy: 0.1958
Epoch 6/6
609/609 [==============================] - 80s 131ms/step - loss: 2.0407 - categorical_accuracy: 0.2432 - val_loss: 2.1390 - val_categorical_accuracy:

609/609 [==============================] - 205s 308ms/step - loss: 2.0157 - categorical_accuracy: 0.2485 - val_loss: 2.1258 - val_categorical_accuracy: 0.2151
Epoch 2/6
609/609 [==============================] - 185s 303ms/step - loss: 1.9896 - categorical_accuracy: 0.2572 - val_loss: 2.1042 - val_categorical_accuracy: 0.2096
Epoch 3/6
609/609 [==============================] - 185s 303ms/step - loss: 1.9563 - categorical_accuracy: 0.2642 - val_loss: 2.0781 - val_categorical_accuracy: 0.2233
Epoch 4/6
609/609 [==============================] - 185s 303ms/step - loss: 1.9456 - categorical_accuracy: 0.2640 - val_loss: 2.1164 - val_categorical_accuracy: 0.2224
Epoch 5/6
609/609 [==============================] - 185s 303ms/step - loss: 1.9211 - categorical_accuracy: 0.2731 - val_loss: 2.0909 - val_categorical_accuracy: 0.2381
Epoch 6/6
609/609 [==============================] - 185s 303ms/step - loss: 1.9165 - categorical_accuracy: 0.2787 - val_loss: 2.0890 - val_categorical_accuracy: 0.2

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.5
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = bert_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    #model2.summary()

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.5


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 135ms/step - loss: 2.2785 - categorical_accuracy: 0.1874 - val_loss: 2.1913 - val_categorical_accuracy: 0.1756
Epoch 2/6
609/609 [==============================] - 80s 131ms/step - loss: 2.0369 - categorical_accuracy: 0.2328 - val_loss: 2.1467 - val_categorical_accuracy: 0.2013
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9804 - categorical_accuracy: 0.2607 - val_loss: 2.0973 - val_categorical_accuracy: 0.2243
Epoch 4/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9389 - categorical_accuracy: 0.2745 - val_loss: 2.0932 - val_categorical_accuracy: 0.2472
Epoch 5/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9070 - categorical_accuracy: 0.2903 - val_loss: 2.0543 - val_categorical_accuracy: 0.2482
Epoch 6/6
609/609 [==============================] - 80s 131ms/step - loss: 1.8789 - categorical_accuracy: 0.2850 - val_loss: 2.0647 - val_categorical_accuracy:

609/609 [==============================] - 205s 308ms/step - loss: 1.8338 - categorical_accuracy: 0.3029 - val_loss: 1.9680 - val_categorical_accuracy: 0.2619
Epoch 2/6
609/609 [==============================] - 185s 304ms/step - loss: 1.7193 - categorical_accuracy: 0.3211 - val_loss: 1.9922 - val_categorical_accuracy: 0.2656
Epoch 3/6
609/609 [==============================] - 185s 304ms/step - loss: 1.6199 - categorical_accuracy: 0.3390 - val_loss: 1.9621 - val_categorical_accuracy: 0.2574
Epoch 4/6
609/609 [==============================] - 185s 304ms/step - loss: 1.5442 - categorical_accuracy: 0.3506 - val_loss: 1.9326 - val_categorical_accuracy: 0.2739
Epoch 5/6
609/609 [==============================] - 185s 304ms/step - loss: 1.4593 - categorical_accuracy: 0.3778 - val_loss: 1.9253 - val_categorical_accuracy: 0.3015
Epoch 6/6
609/609 [==============================] - 185s 304ms/step - loss: 1.3994 - categorical_accuracy: 0.3975 - val_loss: 1.8392 - val_categorical_accuracy: 0.3

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


...

Epoch 1/6
609/609 [==============================] - 94s 135ms/step - loss: 2.5205 - categorical_accuracy: 0.1425 - val_loss: 2.3374 - val_categorical_accuracy: 0.1480
Epoch 2/6
609/609 [==============================] - 79s 130ms/step - loss: 2.3057 - categorical_accuracy: 0.1819 - val_loss: 2.2816 - val_categorical_accuracy: 0.1664
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 2.2312 - categorical_accuracy: 0.2128 - val_loss: 2.2311 - val_categorical_accuracy: 0.1811
Epoch 4/6
609/609 [==============================] - 79s 130ms/step - loss: 2.1599 - categorical_accuracy: 0.2236 - val_loss: 2.2317 - val_categorical_accuracy: 0.1866
Epoch 5/6
609/609 [==============================] - 79s 130ms/step - loss: 2.1126 - categorical_accuracy: 0.2339 - val_loss: 2.2113 - val_categorical_accuracy: 0.1847
Epoch 6/6
609/609 [==============================] - 79s 130ms/step - loss: 2.0773 - categorical_accuracy: 0.2416 - val_loss: 2.1986 - val_categorical_accuracy:

609/609 [==============================] - 204s 308ms/step - loss: 2.0316 - categorical_accuracy: 0.2578 - val_loss: 2.2454 - val_categorical_accuracy: 0.1811
Epoch 2/6
609/609 [==============================] - 185s 304ms/step - loss: 1.9390 - categorical_accuracy: 0.2811 - val_loss: 1.9728 - val_categorical_accuracy: 0.2969
Epoch 3/6
609/609 [==============================] - 185s 304ms/step - loss: 1.8722 - categorical_accuracy: 0.3072 - val_loss: 2.1670 - val_categorical_accuracy: 0.2289
Epoch 4/6
609/609 [==============================] - 185s 304ms/step - loss: 1.8112 - categorical_accuracy: 0.3215 - val_loss: 2.1646 - val_categorical_accuracy: 0.2335
Epoch 5/6
609/609 [==============================] - 185s 304ms/step - loss: 1.7418 - categorical_accuracy: 0.3276 - val_loss: 2.0594 - val_categorical_accuracy: 0.2757
              precision    recall  f1-score   support

         0.0       0.14      0.19      0.16       190
         1.0       0.19      0.50      0.28       173
  

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 135ms/step - loss: 2.5813 - categorical_accuracy: 0.1196 - val_loss: 2.2888 - val_categorical_accuracy: 0.1287
Epoch 2/6
609/609 [==============================] - 79s 131ms/step - loss: 2.3399 - categorical_accuracy: 0.1607 - val_loss: 2.2221 - val_categorical_accuracy: 0.1535
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 2.2385 - categorical_accuracy: 0.1825 - val_loss: 2.1884 - val_categorical_accuracy: 0.1700
Epoch 4/6
609/609 [==============================] - 80s 131ms/step - loss: 2.1680 - categorical_accuracy: 0.2026 - val_loss: 2.1762 - val_categorical_accuracy: 0.1765
Epoch 5/6
609/609 [==============================] - 79s 130ms/step - loss: 2.1307 - categorical_accuracy: 0.2115 - val_loss: 2.1658 - val_categorical_accuracy: 0.1829
Epoch 6/6
609/609 [==============================] - 80s 131ms/step - loss: 2.1001 - categorical_accuracy: 0.2184 - val_loss: 2.1603 - val_categorical_accuracy:

609/609 [==============================] - 205s 308ms/step - loss: 2.0619 - categorical_accuracy: 0.2253 - val_loss: 2.1327 - val_categorical_accuracy: 0.1939
Epoch 2/6
609/609 [==============================] - 185s 304ms/step - loss: 2.0486 - categorical_accuracy: 0.2334 - val_loss: 2.1031 - val_categorical_accuracy: 0.1967
Epoch 3/6
609/609 [==============================] - 185s 304ms/step - loss: 2.0223 - categorical_accuracy: 0.2395 - val_loss: 2.1161 - val_categorical_accuracy: 0.1903
Epoch 4/6
609/609 [==============================] - 185s 304ms/step - loss: 2.0064 - categorical_accuracy: 0.2426 - val_loss: 2.0986 - val_categorical_accuracy: 0.2013
Epoch 5/6
609/609 [==============================] - 185s 304ms/step - loss: 1.9984 - categorical_accuracy: 0.2461 - val_loss: 2.1002 - val_categorical_accuracy: 0.1994
Epoch 6/6
609/609 [==============================] - 185s 304ms/step - loss: 1.9790 - categorical_accuracy: 0.2593 - val_loss: 2.0882 - val_categorical_accuracy: 0.2

In [ ]:
McNemar.keys()

dict_keys([(False, 0.4, 0.0001, 1e-05), (True, 0.4, 0.0001, 1e-05), (False, 0.4, 0.0001, 1e-06), (True, 0.4, 0.0001, 1e-06), (False, 0.5, 0.001, 1e-05), (True, 0.5, 0.001, 1e-05), (False, 0.5, 0.001, 1e-06), (True, 0.5, 0.001, 1e-06), (False, 0.5, 0.0001, 1e-05), (True, 0.5, 0.0001, 1e-05), (False, 0.5, 0.0001, 1e-06), (True, 0.5, 0.0001, 1e-06)])

In [ ]:
df_McNemar_BERT_split0_fine_tune = pd.DataFrame(data=McNemar[(True,0.5, 1e-3, 1e-5)])
df_McNemar_BERT_split0_fine_tune.to_csv(DIR + 'McNemar_BERT_split0_fine_tune.csv')

In [ ]:
df_McNemar_BERT_split0_transfer_learning = pd.DataFrame(data=McNemar[(False,0.5, 1e-3, 1e-5)])
df_McNemar_BERT_split0_transfer_learning.to_csv(DIR + 'McNemar_BERT_split0_transfer_learning.csv')

In [ ]:
balanced_accuracies_fine_tuning

[0.3545180413326312,
 0.32561995928565546,
 0.3198687138125366,
 0.2922621302487614]

In [ ]:
balanced_accuracies_fine_tuning

[0.3545180413326312,
 0.32561995928565546,
 0.3198687138125366,
 0.2922621302487614,
 0.35156416694413817,
 0.3320729042652105,
 0.3001390737921044]

### Best parameter is: drop out rate= 0.5, initial learn rate= 1e-3 , fine tune rate= 1e-5

### KFold test

Best parameter set is: initial learning rate=1e-3, fine tuning learning rate=1e-5, drop out rate=0.2 

In [ ]:
del(dataset_train)
del(dataset_test)
del(train)
del(val)
del(test)

#### Fold1: intentionally separate cells for check point purpose.

In [ ]:
from transformers import AutoTokenizer
import numpy as np

split0=splits[1]
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )
SEQ_LEN=256
print(split0['X_train'].shape, split0['X_test'].shape)

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Xids_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xmask_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']

for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']

print("Xids_train.shape, Xids_test.shape: ",Xids_train.shape, Xids_test.shape)

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))

dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)
dataset_train = dataset_train.shuffle(42).batch(16)

train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(16)

(10832,) (2708,)
Xids_train.shape, Xids_test.shape:  (10832, 256) (2708, 256)


In [ ]:
drop_out_rate = 0.5
learning_rate_transfer_learning =1e-3
learning_rate_fine_tuning = 1e-5

In [ ]:
#if len(balanced_accuracy_transfer_learning)==0:
balanced_accuracy_transfer_learning = []
#if len(balanced_accuracy_fine_tuning)==0:
balanced_accuracy_fine_tune = []

In [ ]:
McNemar.keys()

dict_keys([(False, 0.4, 0.0001, 1e-05), (True, 0.4, 0.0001, 1e-05), (False, 0.4, 0.0001, 1e-06), (True, 0.4, 0.0001, 1e-06), (False, 0.5, 0.001, 1e-05), (True, 0.5, 0.001, 1e-05), (False, 0.5, 0.001, 1e-06), (True, 0.5, 0.001, 1e-06), (False, 0.5, 0.0001, 1e-05), (True, 0.5, 0.0001, 1e-05), (False, 0.5, 0.0001, 1e-06), (True, 0.5, 0.0001, 1e-06)])

In [ ]:
from transformers import TFAutoModel

roberta_base = TFAutoModel.from_pretrained('roberta-base')
SEQ_LEN2=256
#step1
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = roberta_base(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model2.layers[2].trainable = False
print(model2.summary())

#step2
optimizer = tf.keras.optimizers.Adam(learning_rate= learning_rate_transfer_learning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
print(model2.summary() )#Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
#y_pred_transfer_learning=get_balanced_accuracy(model2)
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracy_transfer_learning.append(balanced_acc)


#step5: fine tune
model2.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate_fine_tuning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
#y_pred_transfer_learning=get_balanced_accuracy(model2)
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracy_fine_tune.append(balanced_acc)
del(model2)


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 el)                            thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

609/609 [==============================] - 206s 313ms/step - loss: 2.0690 - categorical_accuracy: 0.2652 - val_loss: 3.2000 - val_categorical_accuracy: 0.0928
Epoch 2/6
609/609 [==============================] - 187s 307ms/step - loss: 2.0267 - categorical_accuracy: 0.2713 - val_loss: 5.2302 - val_categorical_accuracy: 0.0744
Epoch 3/6
609/609 [==============================] - 187s 307ms/step - loss: 1.9901 - categorical_accuracy: 0.2755 - val_loss: 4.3445 - val_categorical_accuracy: 0.1131
Epoch 4/6
609/609 [==============================] - 187s 307ms/step - loss: 1.9398 - categorical_accuracy: 0.2888 - val_loss: 5.0156 - val_categorical_accuracy: 0.0873
Epoch 5/6
609/609 [==============================] - 187s 307ms/step - loss: 1.8871 - categorical_accuracy: 0.2925 - val_loss: 3.3289 - val_categorical_accuracy: 0.1094
Epoch 6/6
609/609 [==============================] - 187s 307ms/step - loss: 1.8355 - categorical_accuracy: 0.3091 - val_loss: 2.8144 - val_categorical_accuracy: 0.1

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

#### Fold2: intentionally separate cells for check point purpose.

In [ ]:
from transformers import AutoTokenizer
import numpy as np

split0=splits[2]
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )
SEQ_LEN=256
print(split0['X_train'].shape, split0['X_test'].shape)

tokenizer = AutoTokenizer.from_pretrained('roberta-base')

Xids_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xmask_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']

for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']

print("Xids_train.shape, Xids_test.shape: ",Xids_train.shape, Xids_test.shape)

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))

dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)
dataset_train = dataset_train.shuffle(42).batch(16)

train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(16)

(10832,) (2708,)


Xids_train.shape, Xids_test.shape:  (10832, 256) (2708, 256)


In [ ]:
drop_out_rate = 0.5
learning_rate_transfer_learning =1e-3
learning_rate_fine_tuning = 1e-5

In [ ]:
if len(balanced_accuracy_transfer_learning)==0:
  balanced_accuracy_transfer_learning = []
if len(balanced_accuracy_fine_tune)==0:
  balanced_accuracy_fine_tune = []

In [ ]:
McNemar.keys()

dict_keys([(False, 0.4, 0.0001, 1e-06), (True, 0.4, 0.0001, 1e-06), (False, 0.5, 0.001, 1e-05), (True, 0.5, 0.001, 1e-05), (False, 0.5, 0.001, 1e-06), (True, 0.5, 0.001, 1e-06), (False, 0.5, 0.0001, 1e-05), (True, 0.5, 0.0001, 1e-05), (False, 0.5, 0.0001, 1e-06), (True, 0.5, 0.0001, 1e-06), (False, 0.2, 0.001, 1e-05), (True, 0.2, 0.001, 1e-05)])

In [ ]:
from transformers import TFAutoModel

roberta_base = TFAutoModel.from_pretrained('roberta-base')
SEQ_LEN2=256
#step1
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = roberta_base(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model2.layers[2].trainable = False
print(model2.summary())

#step2
optimizer = tf.keras.optimizers.Adam(learning_rate= learning_rate_transfer_learning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
print(model2.summary() )#Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
#y_pred_transfer_learning=get_balanced_accuracy(model2)
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracy_transfer_learning.append(balanced_acc)


#step5: fine tune
model2.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate_fine_tuning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
#y_pred_transfer_learning=get_balanced_accuracy(model2)
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracy_fine_tune.append(balanced_acc)
del(model2)


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_roberta_model_1 (TFRobertaM  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 odel)                          thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

609/609 [==============================] - 208s 312ms/step - loss: 1.8701 - categorical_accuracy: 0.2817 - val_loss: 1.7959 - val_categorical_accuracy: 0.3143
Epoch 2/6
609/609 [==============================] - 187s 307ms/step - loss: 1.7823 - categorical_accuracy: 0.2990 - val_loss: 2.1377 - val_categorical_accuracy: 0.2344
Epoch 3/6
609/609 [==============================] - 187s 307ms/step - loss: 1.6984 - categorical_accuracy: 0.3192 - val_loss: 2.0663 - val_categorical_accuracy: 0.3033
Epoch 4/6
609/609 [==============================] - 187s 307ms/step - loss: 1.6316 - categorical_accuracy: 0.3428 - val_loss: 1.7964 - val_categorical_accuracy: 0.3088
              precision    recall  f1-score   support

         0.0       0.23      0.17      0.20       191
         1.0       0.26      0.32      0.29       172
         2.0       0.13      0.07      0.09       134
         3.0       0.29      0.13      0.18       216
         4.0       0.68      0.26      0.37      1036
         

#### Fold3: intentionally separate cells for check point purpose.

In [ ]:
from transformers import AutoTokenizer
import numpy as np

split0=splits[3]
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )
SEQ_LEN=256
print(split0['X_train'].shape, split0['X_test'].shape)

tokenizer = AutoTokenizer.from_pretrained('roberta-base')

Xids_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xmask_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']

for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']

print("Xids_train.shape, Xids_test.shape: ",Xids_train.shape, Xids_test.shape)

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))

dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)
dataset_train = dataset_train.shuffle(42).batch(16)

train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(16)

(10832,) (2708,)
Xids_train.shape, Xids_test.shape:  (10832, 256) (2708, 256)


In [ ]:
drop_out_rate = 0.5
learning_rate_transfer_learning =1e-3
learning_rate_fine_tuning = 1e-5

In [ ]:
if len(balanced_accuracy_transfer_learning)==0:
  balanced_accuracy_transfer_learning = []
if len(balanced_accuracy_fine_tune)==0:
  balanced_accuracy_fine_tune = []

In [ ]:
McNemar.keys()

dict_keys([(False, 0.4, 0.0001, 1e-06), (True, 0.4, 0.0001, 1e-06), (False, 0.5, 0.001, 1e-05), (True, 0.5, 0.001, 1e-05), (False, 0.5, 0.001, 1e-06), (True, 0.5, 0.001, 1e-06), (False, 0.5, 0.0001, 1e-05), (True, 0.5, 0.0001, 1e-05), (False, 0.5, 0.0001, 1e-06), (True, 0.5, 0.0001, 1e-06), (False, 0.2, 0.001, 1e-05), (True, 0.2, 0.001, 1e-05)])

In [ ]:
from transformers import TFAutoModel

roberta_base = TFAutoModel.from_pretrained('roberta-base')
SEQ_LEN2=256
#step1
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = roberta_base(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model2.layers[2].trainable = False
print(model2.summary())

#step2
optimizer = tf.keras.optimizers.Adam(learning_rate= learning_rate_transfer_learning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
print(model2.summary() )#Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
#y_pred_transfer_learning=get_balanced_accuracy(model2)
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracy_transfer_learning.append(balanced_acc)


#step5: fine tune
model2.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate_fine_tuning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
#y_pred_transfer_learning=get_balanced_accuracy(model2)
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracy_fine_tune.append(balanced_acc)
del(model2)


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_roberta_model_2 (TFRobertaM  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 odel)                          thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

609/609 [==============================] - 207s 311ms/step - loss: 1.8725 - categorical_accuracy: 0.2869 - val_loss: 2.1187 - val_categorical_accuracy: 0.2445
Epoch 2/6
609/609 [==============================] - 186s 306ms/step - loss: 1.8020 - categorical_accuracy: 0.2956 - val_loss: 1.8271 - val_categorical_accuracy: 0.3143
Epoch 3/6
609/609 [==============================] - 186s 306ms/step - loss: 1.7419 - categorical_accuracy: 0.3117 - val_loss: 2.0408 - val_categorical_accuracy: 0.2748
Epoch 4/6
609/609 [==============================] - 186s 306ms/step - loss: 1.6874 - categorical_accuracy: 0.3385 - val_loss: 1.8672 - val_categorical_accuracy: 0.3456
Epoch 5/6
609/609 [==============================] - 186s 306ms/step - loss: 1.5979 - categorical_accuracy: 0.3556 - val_loss: 1.8225 - val_categorical_accuracy: 0.3180
Epoch 6/6
609/609 [==============================] - 186s 306ms/step - loss: 1.5207 - categorical_accuracy: 0.3751 - val_loss: 2.6900 - val_categorical_accuracy: 0.2

#### Fold4: intentionally separate cells for check point purpose.

In [ ]:
from transformers import AutoTokenizer
import numpy as np

split0=splits[4]
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )
SEQ_LEN=256
print(split0['X_train'].shape, split0['X_test'].shape)

tokenizer = AutoTokenizer.from_pretrained('roberta-base')

Xids_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xmask_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']

for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']

print("Xids_train.shape, Xids_test.shape: ",Xids_train.shape, Xids_test.shape)

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))

dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)
dataset_train = dataset_train.shuffle(42).batch(16)

train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(16)

(10832,) (2708,)
Xids_train.shape, Xids_test.shape:  (10832, 256) (2708, 256)


In [ ]:
drop_out_rate = 0.5
learning_rate_transfer_learning =1e-3
learning_rate_fine_tuning = 1e-5

In [ ]:
if len(balanced_accuracy_transfer_learning)==0:
  balanced_accuracy_transfer_learning = []
if len(balanced_accuracy_fine_tune)==0:
  balanced_accuracy_fine_tune = []

In [ ]:
McNemar.keys()

dict_keys([(False, 0.4, 0.0001, 1e-06), (True, 0.4, 0.0001, 1e-06), (False, 0.5, 0.001, 1e-05), (True, 0.5, 0.001, 1e-05), (False, 0.5, 0.001, 1e-06), (True, 0.5, 0.001, 1e-06), (False, 0.5, 0.0001, 1e-05), (True, 0.5, 0.0001, 1e-05), (False, 0.5, 0.0001, 1e-06), (True, 0.5, 0.0001, 1e-06), (False, 0.2, 0.001, 1e-05), (True, 0.2, 0.001, 1e-05)])

In [ ]:
from transformers import TFAutoModel

roberta_base = TFAutoModel.from_pretrained('roberta-base')
SEQ_LEN2=256
#step1
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = roberta_base(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model2.layers[2].trainable = False
print(model2.summary())

#step2
optimizer = tf.keras.optimizers.Adam(learning_rate= learning_rate_transfer_learning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
print(model2.summary() )#Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
#y_pred_transfer_learning=get_balanced_accuracy(model2)
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracy_transfer_learning.append(balanced_acc)


#step5: fine tune
model2.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate_fine_tuning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
#y_pred_transfer_learning=get_balanced_accuracy(model2)
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracy_fine_tune.append(balanced_acc)
del(model2)


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_roberta_model_3 (TFRobertaM  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 odel)                          thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

609/609 [==============================] - 207s 311ms/step - loss: 1.8802 - categorical_accuracy: 0.2860 - val_loss: 1.8673 - val_categorical_accuracy: 0.3070
Epoch 2/6
609/609 [==============================] - 187s 306ms/step - loss: 1.7863 - categorical_accuracy: 0.3150 - val_loss: 1.9109 - val_categorical_accuracy: 0.3208
Epoch 3/6
609/609 [==============================] - 187s 306ms/step - loss: 1.7393 - categorical_accuracy: 0.3206 - val_loss: 1.9823 - val_categorical_accuracy: 0.2877
Epoch 4/6
609/609 [==============================] - 186s 306ms/step - loss: 1.6536 - categorical_accuracy: 0.3501 - val_loss: 1.7699 - val_categorical_accuracy: 0.3153
Epoch 5/6
609/609 [==============================] - 186s 306ms/step - loss: 1.5818 - categorical_accuracy: 0.3625 - val_loss: 1.8261 - val_categorical_accuracy: 0.3189
              precision    recall  f1-score   support

         0.0       0.21      0.36      0.27       190
         1.0       0.22      0.55      0.32       173
  

Take average

In [ ]:
balanced_accuracy_fine_tune

[0.26625640318037086,
 0.3447727236337791,
 0.27716701127652676,
 0.3789276338922123]

In [ ]:
balanced_accuracy_fine_tune.append(0.3560173405159383)

In [ ]:
sum(balanced_accuracy_fine_tune)/5

0.3503256162888067

### Conduct McNemar testing with the best parameter

In [ ]:
drop_out_rate = 0.2
learning_rate_transfer_learning = 1e-3
learning_rate_fine_tuning = 1e-5
McNemar = {}# (is_fine_tuning?, dropout_rate, learning_rate_transfer_learning, learning_rate_fine_tuning)

In [ ]:
from transformers import TFAutoModel

balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256

drop_out_rate = 0.2
learning_rate_transfer_learning = 1e-3
learning_rate_fine_tuning = 1e-5


print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
, learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
#step1
roberta = TFAutoModel.from_pretrained('roberta-base')
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = roberta(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model2.layers[2].trainable = False
#model2.summary()

#step2
optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
#model2.summary() #Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracies_transfer_learning.append( balanced_acc )

#step5: fine tune
print("Fine tuning---------------")
model2.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracies_fine_tuning.append( balanced_acc )
print("----------------------------------------")
del(model2)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.2


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 158s 234ms/step - loss: 2.1384 - categorical_accuracy: 0.2122 - val_loss: 2.8267 - val_categorical_accuracy: 0.0901
Epoch 2/6
609/609 [==============================] - 138s 227ms/step - loss: 1.9793 - categorical_accuracy: 0.2636 - val_loss: 2.3489 - val_categorical_accuracy: 0.1480
Epoch 3/6
609/609 [==============================] - 138s 227ms/step - loss: 1.9232 - categorical_accuracy: 0.2825 - val_loss: 2.4198 - val_categorical_accuracy: 0.1379
Epoch 4/6
609/609 [==============================] - 138s 227ms/step - loss: 1.8901 - categorical_accuracy: 0.2974 - val_loss: 2.3096 - val_categorical_accuracy: 0.1581
Epoch 5/6
609/609 [==============================] - 138s 227ms/step - loss: 1.8674 - categorical_accuracy: 0.2979 - val_loss: 2.2364 - val_categorical_accuracy: 0.1801
Epoch 6/6
609/609 [==============================] - 138s 227ms/step - loss: 1.8519 - categorical_accuracy: 0.3018 - val_loss: 2.2021 - val_categorical_acc

609/609 [==============================] - 357s 553ms/step - loss: 1.7825 - categorical_accuracy: 0.3348 - val_loss: 1.8293 - val_categorical_accuracy: 0.2960
Epoch 2/6
609/609 [==============================] - 333s 547ms/step - loss: 1.6608 - categorical_accuracy: 0.3509 - val_loss: 1.8948 - val_categorical_accuracy: 0.2923
Epoch 3/6
609/609 [==============================] - 333s 547ms/step - loss: 1.5689 - categorical_accuracy: 0.3829 - val_loss: 2.0517 - val_categorical_accuracy: 0.2491
Epoch 4/6
609/609 [==============================] - 333s 547ms/step - loss: 1.4769 - categorical_accuracy: 0.4039 - val_loss: 1.9109 - val_categorical_accuracy: 0.2978
Epoch 5/6
609/609 [==============================] - 333s 547ms/step - loss: 1.3818 - categorical_accuracy: 0.4348 - val_loss: 1.7944 - val_categorical_accuracy: 0.3529
Epoch 6/6
609/609 [==============================] - 333s 547ms/step - loss: 1.2942 - categorical_accuracy: 0.4637 - val_loss: 1.7972 - val_categorical_accuracy: 0.3

In [ ]:
McNemar_RoBERTa_split0_transfer_learning = pd.DataFrame(data= McNemar[(False,0.2, 1e-3,1e-5)] )
McNemar_RoBERTa_split0_transfer_learning.to_csv(DIR + 'McNemar_RoBERTa_split0_transfer_learning.csv')

McNemar_RoBERTa_split0_fine_tuning = pd.DataFrame(data= McNemar[(True, 0.2, 1e-3,1e-5)] )
McNemar_RoBERTa_split0_fine_tuning.to_csv(DIR + 'McNemar_RoBERTa_split0_fine_tuning.csv')